Generar código QR 

In [20]:
import qrcode
import json

data = {
    "salon" : "S501"
}

qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)


qr.add_data(json.dumps(data)) 
qr.make(fit=True)

img = qr.make_image(fill='black', back_color='white')

img.save("data/codigo_qr.png")

In [2]:
from pymongo import MongoClient

# Conexión a MongoDB en localhost
client = MongoClient("mongodb://localhost:27017/")

# Seleccionar la base de datos
db = client["QR_Scanner"]

# Seleccionar la colección
asistencias = db["Asistencia"]

# Insertar un documento de prueba
asistencias.insert_one({
    "id_salon": "S501",
    "id_curso": "C101",
    "docente": "D2001",
    "fecha": "2025-02-06T10:00:00",
    "estudiantes": []
})

print("Documento insertado con éxito.")


Documento insertado con éxito.


## Conexión Mongo

In [1]:
from pymongo import MongoClient
from datetime import datetime

# Conectarse a MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["QR_Scanner"]
asistencias = db["Asistencia"]

## Traer data de cursos

In [ ]:

import locale

cursos = db["Cursos"]
locale.setlocale(locale.LC_TIME, 'es_ES.utf8')  

# Parámetros de búsqueda
salon = "S501"
hora_actual = datetime.now().strftime("%H:%M:%S")  # Obtener la hora actual en formato HH:MM:SS
dia_actual = datetime.today().strftime("%A").lower()  # Obtener el día en español

# Buscar el curso activo en ese salón y horario
curso_actual = cursos.find_one({
    "id_salon": salon,
    "horario": {
        "$elemMatch": {
            "dia": dia_actual,
            "hora_inicio": {"$lte": hora_actual},  # La clase ya inició
            "hora_fin": {"$gte": hora_actual}      # La clase aún no ha terminado
        }
    }
})

# Mostrar el curso activo
if curso_actual:
    print(f"Curso activo en {salon}: {curso_actual['nombre']} con {curso_actual['docente']}")
else:
    print(f"No hay curso activo en el salón {salon} en este momento.")


Curso activo en S501: Matemáticas Avanzadas con D2001


## Traer data usuario

In [4]:
from pymongo import MongoClient

correo_a_buscar = "jdalvarezp@udistrital.edu.co"

docente = db["Docentes"].find_one({"correo": correo_a_buscar})
if docente:
    print({"tipo": "docente", "datos": docente})
else:
    estudiante = db["Estudiantes"].find_one({"correo": correo_a_buscar})
    if estudiante:
        print({"tipo": "estudiante", "datos": estudiante})
    else:
        print("No encontrado")


{'tipo': 'docente', 'datos': {'_id': 'D001', 'nombre': 'José David Alvarez Plata', 'correo': 'jdalvarezp@udistrital.edu.co', 'contraseña': 1234, 'tipo_usuario': 'docente', 'modalidad': 'planta', 'estado': 'activo'}}


In [22]:
from datetime import datetime

datetime.now().strftime("%H:%M:%S")  

'22:43:09'

In [18]:
datetime.now().strftime("%Y-%m-%d")

'2025-02-07'

In [19]:
import locale
from datetime import datetime

# Configurar el idioma a español
#locale.setlocale(locale.LC_TIME, 'es_ES.utf8')  # En Linux/Mac
locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')  # En Windows

# Obtener el nombre del día en español
nombre_dia = datetime.today().strftime('%A')

print(nombre_dia)  # Ejemplo: 'miércoles'


sábado


## Insertar un documento

In [10]:

# Insertar un documento de prueba
asistencias.insert_one({
    "id_salon": "S501",
    "id_curso": "C101",
    "docente": "D2001",
    "fecha": "2025-02-07",
    "estudiantes": []
})

print("Documento insertado con éxito.")


Documento insertado con éxito.


## Login

In [15]:
import tkinter as tk
from tkinter import messagebox
from pymongo import MongoClient

class LoginApp:
    def __init__(self):
        # Conectar a MongoDB
        self.client = MongoClient("mongodb://localhost:27017")
        self.db = self.client["QR_Scanner"]  # Reemplaza con el nombre de tu base de datos
        self.docentes_collection = self.db["Docentes"]  # Reemplaza con el nombre de tu colección de docentes
        self.estudiantes_collection = self.db["Estudiantes"]  # Reemplaza con la colección de estudiantes

        self.usuario = None
        self.ventana = tk.Tk()
        self.ventana.title("Login QR Scanner")
        self.ventana.geometry("400x600")
        self.ventana.config(bg="#f2f2f2")

        titulo = tk.Label(self.ventana, text="Iniciar sesión", font=("Arial", 24, "bold"), bg="#f2f2f2")
        titulo.pack(pady=30)

        label_usuario = tk.Label(self.ventana, text="Usuario", font=("Arial", 14), bg="#f2f2f2")
        label_usuario.pack(pady=5)

        self.entry_usuario = tk.Entry(self.ventana, font=("Arial", 14), width=25, bd=2, relief="solid")
        self.entry_usuario.pack(pady=10)

        label_contraseña = tk.Label(self.ventana, text="Contraseña", font=("Arial", 14), bg="#f2f2f2")
        label_contraseña.pack(pady=5)

        self.entry_contraseña = tk.Entry(self.ventana, font=("Arial", 14), show="*", width=25, bd=2, relief="solid")
        self.entry_contraseña.pack(pady=10)

        boton_login = tk.Button(self.ventana, text="Ingresar", font=("Arial", 12), bg="#4CAF50", fg="white", width=25, height=2, 
                                command=self.realizar_login)
        boton_login.pack(pady=20)

        boton_outlook = tk.Button(self.ventana, text="Ingresar con Outlook", font=("Arial", 12), bg="#0078D4", fg="white", width=25, height=2, 
                                  command=self.login_outlook)
        boton_outlook.pack(pady=10)

        self.ventana.mainloop()

    def realizar_login(self):
        usuario = self.entry_usuario.get()
        contraseña = self.entry_contraseña.get()

        # Buscar en docentes
        usuario_db = self.docentes_collection.find_one({"correo": usuario})
        tipo_usuario = "docente"

        # Si no está en docentes, buscar en estudiantes
        if not usuario_db:
            usuario_db = self.estudiantes_collection.find_one({"correo": usuario})
            tipo_usuario = "estudiante"

        # Validar credenciales
        if usuario_db and usuario_db["contraseña"] == contraseña:
            messagebox.showinfo("Login exitoso", f"¡Bienvenido {usuario} ({tipo_usuario})!")
            self.usuario = usuario
            self.ventana.destroy()
        else:
            messagebox.showerror("Error", "Usuario o contraseña incorrectos.")

    def login_outlook(self):
        messagebox.showinfo("Login Outlook", "Ingreso con Outlook simulado.")

def ejecutar_login():
    app = LoginApp()
    return app.usuario

usuario = ejecutar_login()